In [10]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType
import json
import pyspark
from pyspark.sql.functions import from_json, col
from datetime import datetime
import time

In [2]:
spark = SparkSession.builder.master("local").appName("FraudCheck").getOrCreate()

22/12/07 10:49:34 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.124.1 instead (on interface eth1)
22/12/07 10:49:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/07 10:49:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("header","true").csv("DB/client.csv")

In [4]:
banned_countries = ['North Korea', 'Russia', 'Iran', 'Cuba', 'Sudan', 'Syria', 'Belgique']

In [13]:
def login_check(value):
    if value["country"] in banned_countries:
        return False
    return True

def check(row, key, expected):
    return row[key] == expected
    
def credit_card_check(value):
    
    reason = "Forbidden Country"
    if not login_check(value):
        return False, reason
        
    reason = "False Card"
    name = value["name"]
    number = value["number"]
    CVC = value["CVC"]
    expire = value["expire"]
    row = df.where(df.number==number).limit(1).collect()
    
    if len(row) == 0:
        return False, reason
    row = row[0]
    res = check(row, "number", number) and check(row, "CVC", CVC) and check(row, "expire", expire) and check(row, "name", name)
    if res:
        reason = "Card accepted"
    return res, reason
    
#credit_card_check({"name":'Stephanie Vega', "number":'180039745386698', "CVC":'187', "expire":'02/31'})

In [ ]:
def send(msg, producer, topic):
    msg = json.dumps(msg)        
    producer.send(topic, msg.encode('utf-8'))
    producer.flush()
    

producer = KafkaProducer(bootstrap_servers="localhost")

consumer = KafkaConsumer('t3')

for msg in consumer:
   value = json.loads(msg.value)
   res, reason = credit_card_check(value)
   print(value)
   time.sleep(0.1)
   send({"ID" : value["ID"], "res": str(res), "body": reason}, producer, "t4")        

{'date': '2022/12/07 14:40:35', 'ID': 'ba17f3d2-7634-11ed-b861-309c23ff59e3', 'country': 'France', 'name': 'a', 'number': 'a', 'CVC': 'aa', 'expire': 'a'}
